In [ ]:
pip install underthesea

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import re
from underthesea import text_normalize, word_tokenize

In [ ]:
df_train=pd.read_csv('./ViCTSD_train.csv')
df_dev=pd.read_csv('./ViCTSD_valid.csv')
df_test=pd.read_csv('./ViCTSD_test.csv')

In [ ]:
df_train

,Unnamed: 0,Comment,Constructiveness,Toxicity,Title,Topic
0,6326,Thật tuyệt vời...!!!,0,0,Những 'bước tiến diệu kỳ' của Trúc Nhi - Diệu Nhi,SucKhoe
1,7835,"mỹ đã tuột dốc quá nhiều rồi, giờ muốn vực dậy...",1,0,Hình tượng Mỹ sụp đổ trong lòng người dân thế ...,TheGioi
2,4690,tôi thấy người lái xe hơi bấm còi mới là người...,1,1,Cả trăm người đạp xe thể dục bịt kín đường,OtoXemay
3,6011,Coi dịch là giặc. Đã mang tên đó mà xâm nhập V...,0,0,11 ngày không lây nhiễm nCoV cộng đồng,SucKhoe
4,9303,Thương các bé quá! Các con còn quá nhỏ mà đã p...,0,0,5 trẻ chết đuối dưới ao,ThoiSu
...,...,...,...,...,...,...
6995,9296,"đọc tin mà đau lòng quá,thảm nạn cứ xãy ra hoà...",0,0,5 trẻ chết đuối dưới ao,ThoiSu
6996,3645,Nên mua bạn nhé. Để bảo vệ sức khỏe bạn trong ...,1,0,Chưa lập gia đình có nên mua bảo hiểm nhân thọ?,KinhDoanh
6997,5008,Nên ra luật người sử dụng ma túy cũng phạm tội...,1,0,102 người dương tính ma túy trong quán bar,PhapLuat
6998,4343,Mua Fadil đủ lăn bánh. Nếu thấy đắt thì đi Mor...,1,0,450 triệu mua xe gì phục vụ gia đình?,OtoXemay


In [ ]:
for df in [df_train, df_dev, df_test]:
    if 'Comment' in df.columns:
        df.rename(columns={'Comment': 'content'}, inplace=True)

In [ ]:
df_misspell = pd.read_csv('./vietnamese-misspell.csv')
misspell_dict = dict(zip(df_misspell['wrong'], df_misspell['right']))

In [ ]:
emoticon_dict = {
    ":)": "☺️",
    ":))": "☺️",
    ":)))": "☺️",
    "=))": "😄",
    "=)": "😄",
    ":D": "😀",
    ":DD": "😁",
    ":<": "😞",
    ":(": "☹️",
    ":'(": "😢",
    ":'(": "😢",
    ":')": "😂",
    ":')": "😂",
    "<3": "❤️",
    "</3": "💔",
    ":P": "😛",
    ":p": "😛",
    ":O": "😲",
    ":o": "😲",
    ";)": "😉",
    ";-)": "😉",
    ":3": "😺",
    ":^)": "😊",
    "^_^": "😊",
    "-_-": "😑",
    ">_<": "😣",
    "XD": "😆",
    "xD": "😆",
    "T_T": "😭",
    ";_;": "😭",
    ":|": "😐",
    ":/": "😕",
    ":-/": "😕",
    ":-\\": "😕",
    ":'D": "😆",
    ":'D": "😆",
    ":-*": "😘",
    ":*": "😘",
    "<<": "😓",
    ">_>": "😒",
    "<_<": "😒",
    "^\\^": "😆",
    "\\^_^/": "🎉",
    "*^_^*": "🎉",
    "\\o/": "🙌",
    "\\O/": "🙌",
    "O_O": "😳",
    "o_o": "😳",
    ">:O": "😠",
    "^^": "☺️"
}


In [ ]:
def standardize_emoticon(text):
    # Gộp các chuỗi emoticon thường gặp về chuẩn (gộp lặp, ví dụ ^^ ^^ ^^ -> ^^)
    # Gộp emoticon kiểu :) về 1
    text = re.sub(r'((:\)+))', ':)', text)
    text = re.sub(r'((=\)+))', '=)', text)
    text = re.sub(r'((\^_?\^)+)', '^^', text)        # ^^, ^_^, ^^ ^^, ...
    text = re.sub(r'(<3+)', '<3', text)
    text = re.sub(r'(\)+)', ')', text)
    text = re.sub(r'(\(+)', '(', text)
    # Gộp mọi chuỗi ^^ liên tiếp về 1 ^^ (kể cả có cách ra)
    text = re.sub(r'(\^\^)(\s+\^\^)+', '^^', text)
    # Gộp ((
    text = re.sub(r'(\(+)', '(', text)
    text = re.sub(r'(\)+)', ')', text)
    return text

In [ ]:
def convert_emoticon(text, emoticon_dict):
    # Duyệt emoticon dài trước
    for emo in sorted(emoticon_dict, key=len, reverse=True):
        # Chỉ thay thế khi là nguyên một cụm (dùng word boundary nếu cần)
        text = re.sub(re.escape(emo) + r'(?=\s|$)', emoticon_dict[emo], text)
    return text

In [ ]:
def standardize_word(text, misspell_dict):
    # Gộp ký tự kéo dài (đẹpppp -> đẹp)
    text = re.sub(r'(\w)\1{2,}', r'\1', text)
    # Chuẩn hóa chính tả/viết tắt
    for wrong, right in misspell_dict.items():
        text = re.sub(r'\b' + re.escape(wrong) + r'\b', right, text)
    return text

In [ ]:
teencode_map = {}
with open("./vietnamese-teencode.txt", "r", encoding="utf8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 2:
            teencode_map[parts[0]] = parts[1]

In [ ]:
import re

def remove_duplicate_emoji(text):
    # Xóa emoji trùng lặp liên tiếp (vd: 😄😄😄 -> 😄)
    emoji_pattern = re.compile(
        r'([\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF])\1+'
    )
    return emoji_pattern.sub(r'\1', text)


In [ ]:
def preprocess_text(text, emoticon_dict, misspell_dict):
    text = str(text)
    text = standardize_emoticon(text)
    text = convert_emoticon(text, emoticon_dict)
    text = remove_duplicate_emoji(text)
    for abbr, full in teencode_map.items():
        pattern = rf"\b{re.escape(abbr)}\b"
        text = re.sub(pattern, full, text, flags=re.IGNORECASE)
    text = standardize_word(text, misspell_dict)
    return text

In [ ]:
text = "ước gì sau này về già vẫn có thể như cụ này :)) :)))) :)))"
text = preprocess_text(text, emoticon_dict, misspell_dict)
print(text)

ước gì sau này về già vẫn có thể như cụ này ☺️ ☺️ ☺️


In [ ]:
df_test['content_clean'] = df_test['content'].apply(lambda x: preprocess_text(x, emoticon_dict, misspell_dict))
df_test[['content', 'content_clean']].head(10)

,content,content_clean
0,Người ăn không hết kẻ lần chẳng ra,Người ăn không hết kẻ lần chẳng ra
1,Nhiều người cứ nghĩ đạp xe là văn minh. haizzzz,Nhiều người cứ nghĩ đạp xe là văn minh. haiz
2,Rất văn hoá,Rất văn hoá
3,Đời ta ba mươi đời nó. Mua chiếc xe cũng chỉ p...,Đời ta ba mươi đời nó. Mua chiếc xe z chỉ phục...
4,"Tước bằng lái vĩnh viễn đi. Chạy lếu láo thật,...","Tước bằng lái vĩnh viễn đi. Chạy lếu láo thật,..."
5,Cảm ơn các y bác sĩ,Cảm ơn các y bác sĩ
6,Thật tuyệt vời!,Thật tuyệt vời!
7,Quỷ dữ chứ không phải con người. Thật xót xa c...,Quỷ dữ chứ không phải con người. Thật xót xa c...
8,Rất nguy hiểm nếu không quản lý tôt nguồn gốc ...,Rất nguy hiểm nếu không quản lý tôt nguồn gốc ...
9,"Nghe cũng nghẹn lòng thật, tương lai còn dài, ...","Nghe z nghẹn lòng thật, tương lai còn dài, lỡ ..."


In [ ]:
df_train['content_clean'] = df_train['content'].apply(lambda x: preprocess_text(x, emoticon_dict, misspell_dict))
df_train[['content', 'content_clean']].head(10)

,content,content_clean
0,Thật tuyệt vời...!!!,Thật tuyệt vời...!!!
1,"mỹ đã tuột dốc quá nhiều rồi, giờ muốn vực dậy...","mỹ đã tuột dốc quá nhiều rồi, giờ muốn vực dậy..."
2,tôi thấy người lái xe hơi bấm còi mới là người...,tôi thấy người lái xe hơi bấm còi mới là người...
3,Coi dịch là giặc. Đã mang tên đó mà xâm nhập V...,Coi dịch là giặc. Đã mang tên đó mà xâm nhập v...
4,Thương các bé quá! Các con còn quá nhỏ mà đã p...,Thương các bé quá! Các con còn quá nhỏ mà đã p...
5,Sevila đoạt cúp sau loạt đá luân lưu nha,Sevila đoạt cúp sau loạt đá luân lưu nha
6,Angelina Jolie có đôi bàn tay nổi gân guốc quá!,Angelina Jolie có đôi bàn tay nổi gân guốc quá!
7,"""Tôi không biết bà ấy làm thế nào để xử lý gã ...","""Tôi không biết bà ấy làm thế nào để xử lý gã ..."
8,tấm hình này chắc bộ nhớ máy tính chứa được 1 tấm,tấm hình này chắc bộ nhớ máy tính chứa được 1 tấm
9,Không trung sẽ không còn yên tĩnh,Không trung sẽ không còn yên tĩnh


In [ ]:
df_dev['content_clean'] = df_dev['content'].apply(lambda x: preprocess_text(x, emoticon_dict, misspell_dict))
df_dev[['content', 'content_clean']].head(10)

,content,content_clean
0,Cuộc bầu cư người đứng đầu Quốc gia ở Mỹ là cô...,Cuộc bầu cư người đứng đầu Quốc gia ở Mỹ là cô...
1,Mình thích Messi ở lại để Barca và Cules hiểu ...,Mình thích Messi ở lại để Barca và Cules hiểu ...
2,"dám chui ra,vậy ăn vô lại ếch said :')","dám chui ra,vậy ăn vô lại ếch said 😂"
3,Hoan hô sự quả cảm của các chiến sĩ và sự đồng...,Hoan hô sự quả cảm của các chiến sĩ và sự đồng...
4,Tôi cực lực phản đối cho hs dùng đt trong lớp ...,Tôi cực lực phản đối cho hs dùng điện trong lớ...
5,Ta có công thức tính diện tích tam giác là (cạ...,Ta có công thức tính diện tích tam giác là (cạ...
6,Trước cũng thích con Seltos lắm nhưng giờ nhìn...,Trước z thích con Seltos lắm nhưng giờ nhìn cá...
7,"Chia không đều nhau, không công bằng thì rất d...","Chia không đều nhau, không công bằng thì rất d..."
8,nhìn logo thấy đẹp quá,nhìn logo thấy đẹp quá
9,Không mở đường bên Đồng Nai thì điểm kẹt nằm t...,Không mở đường bên Đồng Nai thì điểm kẹt nằm t...


In [ ]:
df_train.to_csv("./df_train_clean.csv", index=False)
df_dev  .to_csv("./df_dev_clean.csv",   index=False)
df_test .to_csv("./df_test_clean.csv",  index=False)
print("Cleaned files written to /content/")

Cleaned files written to /content/


In [ ]:
text = "dám chui ra,vậy ăn vô lại ếch said :')"
text = preprocess_text(text, emoticon_dict, misspell_dict)
print(text)

dám chui ra,vậy ăn vô lại ếch said 😂
